In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from model import BuildingSimulation as bs, BuildingGraph as bg
from model.utils import *
import matplotlib.colors as mcolors
from model.WallSimulation import convectionDOE2
from runMyBuildingMC import cleanMaterial, getConstructions, setConstructionType
from model.WallSimulation import processMaterials


## DOE-2 Convection

In [ ]:
R_f = pd.Series({
    "Very rough": 2.17,
    "Rough": 1.67,
    "Medium rough": 1.52,
    "Medium smooth": 1.13,
    "Smooth": 1.11,
    "Very smooth": 1.00,
})

V = [0, 2, 4, 6]
h_nat = [1, 2, 3]
plt.figure()
for smoothness, r_f in R_f.items():
    for h in h_nat:
        h_f = convectionDOE2(h, V, r_f)
        plt.plot(V, h_f, label=f"smoothness = {smoothness}, h_nat={h} W/m2K") 
        print(f"smoothness = {smoothness}, h_nat={h} W/m2K: h = {min(h_f)} to {max(h_f)}")
#specify legend outside of plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#make plot tight
plt.tight_layout()
xlabel = plt.xlabel("Wind speed (m/s)")
ylabel = plt.ylabel("Convection coefficient (W/m2K)")



# Checking Constructions

## Default (bad)

In [ ]:
for materialType in ["Light", "Medium", "Heavy"]:
    materials = getConstructions(materialType)
    wallMaterial = processMaterials(materials["wall"], 9)
    partitionMaterial = processMaterials(materials["partition"], 9)
    roofMaterial = processMaterials(materials["roof"], 9)
    floorMaterial = processMaterials(materials["floor"], 9)

    # display(f"{materialType} materials")
    # display(wallMaterial)
    # display(f"total wall R-value in ft²-hr-°F/Btu: {wallMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (wallMaterial['Thermal_Resistance'].sum() * 5.678)}")
    # display(partitionMaterial)
    # display(f"total partition R-value in ft²-hr-°F/Btu: {partitionMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (partitionMaterial['Thermal_Resistance'].sum() * 5.678)}")
    # display(roofMaterial)
    # display(f"total roof R-value in ft²-hr-°F/Btu: {roofMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (roofMaterial['Thermal_Resistance'].sum() * 5.678)}")
    # display(floorMaterial)
    # display(f"total floor R-value (no soil) in ft²-hr-°F/Btu: {floorMaterial['Thermal_Resistance'].iloc[:-1].sum() * 5.678}, U-value in W/m²K: {1 / (floorMaterial['Thermal_Resistance'].iloc[:-1].sum() * 5.678)}")

## My Constructions (good)

In [ ]:
materials = getConstructions("My", constructionFile = "energyPlus/My_Constructions.csv")
wallMaterial = materials["wall"]
partitionMaterial = materials["partition"]
roofMaterial = materials["roof"]
floorMaterial = materials["floor"]

display(wallMaterial)
display(f"total wall R-value in ft²-hr-°F/Btu: {wallMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (wallMaterial['Thermal_Resistance'].sum() * 5.678)}")
display(partitionMaterial)
display(f"total partition R-value in ft²-hr-°F/Btu: {partitionMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (partitionMaterial['Thermal_Resistance'].sum() * 5.678)}")
display(roofMaterial)
display(f"total roof R-value in ft²-hr-°F/Btu: {roofMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (roofMaterial['Thermal_Resistance'].sum() * 5.678)}")
display(floorMaterial)
display(f"total floor R-value (no soil) in ft²-hr-°F/Btu: {floorMaterial['Thermal_Resistance'].iloc[:-1].sum() * 5.678}, U-value in W/m²K: {1 / (floorMaterial['Thermal_Resistance'].iloc[:-1].sum() * 5.678)}")

In [ ]:
for materialType in ["Light", "Medium", "Heavy"]:
    adjustedMaterials = setConstructionType(materials, materialType)
    wallMaterial = adjustedMaterials["wall"]
    partitionMaterial = adjustedMaterials["partition"]
    roofMaterial = adjustedMaterials["roof"]
    floorMaterial = adjustedMaterials["floor"]
    display(wallMaterial)
    display(f"total wall R-value in ft²-hr-°F/Btu: {wallMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (wallMaterial['Thermal_Resistance'].sum() * 5.678)}")
    display(partitionMaterial)
    display(f"total partition R-value in ft²-hr-°F/Btu: {partitionMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (partitionMaterial['Thermal_Resistance'].sum() * 5.678)}")
    display(roofMaterial)
    display(f"total roof R-value in ft²-hr-°F/Btu: {roofMaterial['Thermal_Resistance'].sum() * 5.678}, U-value in W/m²K: {1 / (roofMaterial['Thermal_Resistance'].sum() * 5.678)}")
    display(floorMaterial)
    display(f"total floor R-value (no soil) in ft²-hr-°F/Btu: {floorMaterial['Thermal_Resistance'].iloc[:-1].sum() * 5.678}, U-value in W/m²K: {1 / (floorMaterial['Thermal_Resistance'].iloc[:-1].sum() * 5.678)}")



In [ ]:
processMaterials(floorMaterial, 9)

## Multi Walls

In [ ]:
# constructions  = pd.read_csv("energyPlus/ASHRAE_2005_HOF_Constructions.csv", index_col="Name")
# materials = pd.read_csv("energyPlus/ASHRAE_2005_HOF_Materials.csv", index_col="Name")

In [ ]:
sim_kwargs = {
    "delt": 10,
    "simLength": 96 * 60 * 60,
    "Tout" : 288.15,
    "radG": 400,
    "Tfloor": None,
}
wall_kwargs = {"X": 9, "Y": 10, "material_df": wallMaterial, "h": WallSides(4, 4), "absorptivity" : 0.7, "n": 9}

room_kwargs = {
    "T0": 293.15,
    "V" : 2880, #volume of air
    "Eint" : 250 #internal heat generation
}
vent_kwargs = {
    'H': 1,
    'W' : 1,
    "ventType": "HWP1",
    "alphas": None,
    "As": None,
    "Ls": None
}

connectivityMatrix = np.array([
    [3],
])

roomList = [
    ("R", {}), #room
    ]


In [ ]:
bG = bg.BuildingGraph(connectivityMatrix, roomList)
bG.updateEdges({"wall_kwargs" :wall_kwargs})
bG.updateNodes({
    "room_kwargs": room_kwargs,
    "vent_kwargs": vent_kwargs,
        "rad_kwargs": {"solveType": None},
    })
bG.draw()

build_sim = bs.BuildingSimulation(**sim_kwargs)
build_sim.initialize(bG)

build_sim.run()

Tints = build_sim.bG.G.nodes['R']['Tints']
T_profs = build_sim.bG.G.edges['R', 'R']['T_profs']
wall = build_sim.bG.G.edges['R', 'R']['wall']
h_profs = [4, 8, 12, 16, 20]
h_profs = [h + 48 for h in h_profs]
# h_profs = [0, *h_profs]

# Plotting
plt.figure()
plt.plot(build_sim.hours, Tints, label='Tint')
plt.plot(build_sim.hours, T_profs[0, :], label='T0')
plt.legend()
plt.xlabel('hours')
plt.ylabel('Temperature [K]')

plt.figure()
for h in h_profs:
    i = int(h * 60 * 60 / build_sim.delt)
    plt.plot(wall.x, T_profs[:, i], label=f'{h % 24} hours')

plt.legend()
plt.xlabel('x')
plt.ylabel('Temperature [K]')
plt.show()